# Create Slice for Fabric Rack
This is the slice creation for the ```Trafic Analysis``` tutorial, This is the first step of the assignment, to create the slice on fabric using fablib

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

In [ ]:
slice_name = "TrafficAnalysis"
site = "EDUKY"
print(site)

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10

In [ ]:
network_name = 'lan'

In [ ]:
#A topology with 2 nodes (a client and a server) will be reserved.
try:
    #Create Slice
    slice = fablib.new_slice(name = slice_name)
    
    #Server
    server = slice.add_node(name="Server", site=site)
    server.set_image(image)
    server_port = server.add_component(model=nicmodel, name="server_port").get_interfaces()[0] 
    
    #Client
    client = slice.add_node(name="Client", site=site)
    client.set_image(image)
    client_port = client.add_component(model=nicmodel, name="client_port").get_interfaces()[0] 
    
    #changed name
    lan = slice.add_l2network(name=network_name, interfaces=[server_port, client_port])
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Failed: {e}")

## Continue to The experiment notebook

Once You have completed this notebook you should be able to continue to the Traffic Analysis, you can ether open it on the explorer or click [Here](./Traffic_Analysis.ipynb) to open the next notebook.